In [18]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandasai import SmartDataframe
from pandasai.llm import OpenAI
import seaborn as sns
import warnings
from dotenv import load_dotenv

print(pd.__version__)
warnings.filterwarnings('ignore')
load_dotenv()
sns.set_style()

1.5.3


In [19]:
# api_token = os.getenv('OPEN_AI_API_TOKEN')
# llm = OpenAI(api_token=api_token, model='gpt-4o')

df = pd.read_csv('enrollment_code_introspect.csv', encoding='iso-8859-1')
# data = SmartDataframe(df, config={ "llm": llm, "verbose": True })
df.head()



,enrollment_date,campaign_name,campaign_kind,campaign_end_date,campaign_description,is_ftm,ftm_sai_project,is_cfa,is_carnmpp,is_comet,...,region_name,region_boundary,parent_region_name,parent_region_boundary,plan_name,plan_slug,plan_notes,plan_base_price,organization_name,parent_organization_name
0,2024-01-11,2023 West IL Root Zone Carbon,SustainabilityCampaign,2024-02-29,Enabling sustainability metrics tracking on fi...,True,NaN,False,False,False,...,Western Illinois Division,0106000020E610000001000000010300000001000000A3...,NaN,NaN,NaN,NaN,NaN,NaN,Agrible & Nutrien Projects,]
1,2024-01-11,2023 West IL Root Zone Carbon,SustainabilityCampaign,2024-02-29,Enabling sustainability metrics tracking on fi...,True,NaN,False,False,False,...,Western Illinois Division,0106000020E610000001000000010300000001000000A3...,NaN,NaN,NaN,NaN,NaN,NaN,Agrible & Nutrien Projects,NaN
2,2024-01-11,2023 West IL Root Zone Carbon,SustainabilityCampaign,2024-02-29,Enabling sustainability metrics tracking on fi...,True,NaN,False,False,False,...,Western Illinois Division,0106000020E610000001000000010300000001000000A3...,NaN,NaN,NaN,NaN,NaN,NaN,Agrible & Nutrien Projects,NaN
3,2024-01-11,2023 West IL Root Zone Carbon,SustainabilityCampaign,2024-02-29,Enabling sustainability metrics tracking on fi...,True,NaN,False,False,False,...,Western Illinois Division,0106000020E610000001000000010300000001000000A3...,NaN,NaN,NaN,NaN,NaN,NaN,Agrible & Nutrien Projects,NaN
4,2024-01-11,2023 West IL Root Zone Carbon,SustainabilityCampaign,2024-02-29,Enabling sustainability metrics tracking on fi...,True,NaN,False,False,False,...,Western Illinois Division,0106000020E610000001000000010300000001000000A3...,NaN,NaN,NaN,NaN,NaN,NaN,Agrible & Nutrien Projects,NaN
